![PinballML](https://raw.githubusercontent.com/SmallPlanetiOS/smallplanet_Pinball/master/meta/logo.png?token=AAD0RiQR-zTLfWKOoct9N0PgJ8yEl90Iks5ZuAJ8wA%3D%3D)


# A3C Pinball Model

In [1]:
model_prefix = "tng"

model_codename = "adam"
# Adam, Buy, Charles, David, Edward, Frank, George, Henry, Ida, 
# John, King, Lincoln, Mary, Nora, Ocean, Paul, Queen, Robert, 
# Sam, Tom, Union, Victor, William, Xray, Young, Zebra

model_index = 0  # increments with each model iteration

model_notes = "Full playfield, four temporal layers, 5 actions. Codename: "+model_codename
input_shape = (96,128,3)

model_fileprefix = "%s_%s_%04d" % (model_prefix, model_codename, model_index)
print("Current model filename prefix: " + model_fileprefix)

Current model filename prefix: tng_adam_0000


## Actions
* 0 - NOP
* 1 - Left flipper
* 2 - Right flipper
* 3 - Ball kicker 
* 4 - Right upper flipper

In [0]:
actions = 5


In [3]:
!pip install keras tensorflow coremltools h5py cython
!apt-get -qq install -y python-h5py libhdf5-dev

## Model

In [4]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Model

state = Input(shape=input_shape)
h = Conv2D(16, kernel_size=8, strides=4, activation='relu')(state)    
h = Conv2D(32, kernel_size=4, strides=2, activation='relu')(h)
h = Flatten()(h)
h = Dense(128, activation='relu')(h)

value = Dense(1, activation='linear', name='value')(h)
policy = Dense(actions, activation='softmax', name='policy')(h)

value_network = Model(inputs=state, outputs=value)
policy_network = Model(inputs=state, outputs=policy)
advantage = Input(shape=(1,))
train_network = Model(inputs=[state, advantage], outputs=[value, policy])

Using TensorFlow backend.


In [5]:
value_network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 128, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 31, 16)        3088      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 14, 32)        8224      
_________________________________________________________________
flatten_1 (Flatten)          (None, 4480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               573568    
_________________________________________________________________
value (Dense)                (None, 1)                 129       
Total params: 585,009
Trainable params: 585,009
Non-trainable params: 0
_________________________________________________________________


In [6]:
policy_network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 128, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 31, 16)        3088      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 14, 32)        8224      
_________________________________________________________________
flatten_1 (Flatten)          (None, 4480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               573568    
_________________________________________________________________
policy (Dense)               (None, 5)                 645       
Total params: 585,525
Trainable params: 585,525
Non-trainable params: 0
_________________________________________________________________


## Convert Policy Network to CoreML

In [0]:
keras_filename = model_fileprefix + '.model'
coreml_filename = model_fileprefix + '.mlmodel'

policy_network.save(keras_filename)

In [8]:
import coremltools

# output_labels = ['left','right']
coreml_model = coremltools.converters.keras.convert(keras_filename, 
                                                    input_names='images', 
                                                    image_input_names = 'images')
# coreml_model.author = 'Quinn McHenry'
# coreml_model.short_description = model_notes
# coreml_model.input_description['image'] = 'Composite of four images of the pinball playfield from top-down perspective in grayscale one in each RGBA channel'

coreml_model.save(coreml_filename)

/usr/local/lib/python2.7/dist-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


0 : input_1, <keras.engine.topology.InputLayer object at 0x7f8fe0596c10>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x7f8fe0596cd0>
2 : conv2d_1__activation__, <keras.layers.core.Activation object at 0x7f8fe05969d0>
3 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x7f8fe0596d50>
4 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x7f8fe0596a50>
5 : flatten_1, <keras.layers.core.Flatten object at 0x7f8fe0596e90>
6 : dense_1, <keras.layers.core.Dense object at 0x7f8fe05ab110>
7 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f8fe0596a10>
8 : policy, <keras.layers.core.Dense object at 0x7f8fe05ab210>
9 : policy__activation__, <keras.layers.core.Activation object at 0x7f8fe0596ad0>


In [0]:
from google.colab import files

files.download(coreml_filename)